

- Author: Hemanth
- Date : 22-11-2023







In [ ]:
! pip install transformers
!pip install accelerate

# Orca-2: Teaching Small Language Models How to Reason



The progress from Orca 1 to Orca 2 marks a significant leap in empowering smaller Language Models (LMs) with enhanced reasoning abilities. Orca 1 demonstrated remarkable performance by learning from rich signals, surpassing traditional instruction-tuned models in benchmarks like BigBench Hard and AGIEval.

In our pursuit with Orca 2, we've delved deeper into refining training signals to augment the reasoning capabilities of smaller LMs. The common approach in training smaller LMs often centered on imitation learning, replicating outputs of more powerful models. However, we argue that over-reliance on imitation could potentially constrain the potential of smaller models.

Our focus in Orca 2 has shifted towards teaching these smaller LMs to employ diverse solution strategies tailored to different tasks, distinct from those used by larger models. For instance, while larger models might directly answer complex tasks, smaller models, constrained in capacity, might benefit from alternative approaches.

The innovation lies not merely in imparting various reasoning techniques such as step-by-step analysis, recall-based generation, or direct answering to these smaller models, but in enabling them to discern the most effective strategy for each task independently.

Evaluating Orca 2 involved a meticulous examination across 15 diverse benchmarks, encompassing roughly 100 tasks and over 36,000 distinct prompts. The remarkable outcome was Orca 2's substantial outperformance of models of similar sizes, even achieving comparable or superior performance levels to models 5-10 times larger. Particularly notable was its prowess in complex tasks that tested advanced reasoning abilities in zero-shot settings.

To foster further advancements and encourage collaborative research, we've taken the step to open-source Orca 2. This initiative aims to stimulate ongoing exploration, evaluation, and alignment of smaller Language Models, ushering in a new era of scientific innovation and development in the field.

In [ ]:
import torch
import transformers

if torch.cuda.is_available():
    torch.set_default_device("cuda")
else:
    torch.set_default_device("cpu")

model = transformers.AutoModelForCausalLM.from_pretrained("microsoft/Orca-2-13b" )

# https://github.com/huggingface/transformers/issues/27132
# please use the slow tokenizer since fast and slow tokenizer produces different tokens
tokenizer = transformers.AutoTokenizer.from_pretrained(
        "microsoft/Orca-2-13b",
        use_fast=False,
    )

system_message = "You are Orca, an AI language model created by Microsoft. You are a cautious assistant. You carefully follow instructions. You are helpful and harmless and you follow ethical guidelines and promote positive behavior."
user_message = "How can you determine if a restaurant is popular among locals or mainly attracts tourists, and why might this information be useful?"

prompt = f"<|im_start|>system\n{system_message}<|im_end|>\n<|im_start|>user\n{user_message}<|im_end|>\n<|im_start|>assistant"

inputs = tokenizer(prompt, return_tensors='pt')
output_ids = model.generate(inputs["input_ids"],)
answer = tokenizer.batch_decode(output_ids)[0]

print(answer)

# This example continues showing how to add a second turn message by the user to the conversation
second_turn_user_message = "Give me a list of the key points of your first answer."

# we set add_special_tokens=False because we dont want to automatically add a bos_token between messages
second_turn_message_in_markup = f"\n<|im_start|>user\n{second_turn_user_message}<|im_end|>\n<|im_start|>assistant"
second_turn_tokens = tokenizer(second_turn_message_in_markup, return_tensors='pt', add_special_tokens=False)
second_turn_input = torch.cat([output_ids, second_turn_tokens['input_ids']], dim=1)

output_ids_2 = model.generate(second_turn_input,)
second_turn_answer = tokenizer.batch_decode(output_ids_2)[0]

print(second_turn_answer)


In [ ]:
import os
import math
import transformers
import torch

from azure.ai.contentsafety import ContentSafetyClient
from azure.core.credentials import AzureKeyCredential
from azure.core.exceptions import HttpResponseError
from azure.ai.contentsafety.models import AnalyzeTextOptions

CONTENT_SAFETY_KEY = os.environ["CONTENT_SAFETY_KEY"]
CONTENT_SAFETY_ENDPOINT = os.environ["CONTENT_SAFETY_ENDPOINT"]

# We use Azure AI Content Safety to filter out any content that reaches "Medium" threshold
# For more information: https://learn.microsoft.com/en-us/azure/ai-services/content-safety/
def should_filter_out(input_text, threshold=4):
    # Create an Content Safety client
    client = ContentSafetyClient(CONTENT_SAFETY_ENDPOINT, AzureKeyCredential(CONTENT_SAFETY_KEY))

    # Construct a request
    request = AnalyzeTextOptions(text=input_text)

    # Analyze text
    try:
        response = client.analyze_text(request)
    except HttpResponseError as e:
        print("Analyze text failed.")
        if e.error:
            print(f"Error code: {e.error.code}")
            print(f"Error message: {e.error.message}")
            raise
        print(e)
        raise

    categories = ["hate_result", "self_harm_result", "sexual_result", "violence_result"]
    max_score = -math.inf
    for category in categories:
        max_score = max(max_score, getattr(response, category).severity)

    return max_score >= threshold

model_path = 'microsoft/Orca-2-13b'
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = transformers.AutoModelForCausalLM.from_pretrained(model_path)
model.to(device)

tokenizer = transformers.AutoTokenizer.from_pretrained(
    model_path,
    model_max_length=4096,
    padding_side="right",
    use_fast=False,
    add_special_tokens=False,
)

system_message = "You are Orca, an AI language model created by Microsoft. You are a cautious assistant. You carefully follow instructions. You are helpful and harmless and you follow ethical guidelines and promote positive behavior."
user_message = "\" \n :You can't just say, \"\"that's crap\"\" and remove it without gaining a consensus. You already know this, based on your block history. —/ \" \nIs the comment obscene? \nOptions : Yes, No."

prompt =  f"<|im_start|>system\n{system_message}<|im_end|>\n<|im_start|>user\n{user_message}<|im_end|>\n<|im_start|>assistant"

inputs = tokenizer(prompt, return_tensors='pt')
inputs = inputs.to(device)

output_ids = model.generate(inputs["input_ids"], max_length=4096, do_sample=False, temperature=0.0, use_cache=True)
sequence_length = inputs["input_ids"].shape[1]
new_output_ids = output_ids[:, sequence_length:]
answers = tokenizer.batch_decode(new_output_ids, skip_special_tokens=True)
final_output = answers[0] if not should_filter_out(answers[0]) else "[Content Filtered]"

print(final_output)
